In [ ]:
import numpy as np
import seaborn as sns
from statsmodels.tools.tools import maybe_unwrap_results
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from typing import Type
import statsmodels.formula.api as smf
import patchworklib as pw

In [ ]:
oystercatcher_py = pd.read_csv("data/CS2-oystercatcher-feeding.csv")

In [ ]:
# define the model
model = smf.ols(formula= "feeding ~ C(site)", data = oystercatcher_py)
# fit the model
results = model.fit()

results.summary()

In [ ]:
# Get different Variables for diagnostic
residuals = results.resid.rename("residuals")
fitted_values = results.fittedvalues.rename("fitted_values")
std_resid = pd.Series(results.resid_pearson).rename("std_resid")
influence = results.get_influence()
cooks_d = pd.Series(influence.cooks_distance[0]).rename("cooks_d")
leverage = pd.Series(influence.hat_matrix_diag).rename("leverage")
obs = pd.Series(range(len(residuals))).rename("obs")
n_obs = len(obs.index)

In [ ]:
# combine Series into DataFrame
model_values = residuals.to_frame().join(fitted_values).join(std_resid).join(cooks_d).join(leverage).join(obs)

model_values["n_obs"] = n_obs

In [ ]:
import patchworklib as pw
from plotnine import *
from plotnine.data import *
g1 = (ggplot(mtcars) + geom_point(aes("mpg", "disp")))
g2 = (ggplot(mtcars) + geom_boxplot(aes("gear", "disp", group="gear")))
g3 = (ggplot(mtcars, aes('wt', 'mpg', color='factor(gear)')) + geom_point() + stat_smooth(method='lm') + facet_wrap('~gear'))
g4 = (ggplot(data=diamonds) + geom_bar(mapping=aes(x="cut", fill="clarity"), position="dodge"))

g1 = pw.load_ggplot(g1, figsize=(2,3))
g2 = pw.load_ggplot(g2, figsize=(2,3))
g3 = pw.load_ggplot(g3, figsize=(3,3))
g4 = pw.load_ggplot(g4, figsize=(5,2))
g1234 = (g1|g2|g3)/g4
g1234.savefig()

## Q-Q plot

In [ ]:
p1 = (
  ggplot(model_values, aes(sample = "residuals"))
  + stat_qq()
  + stat_qq_line(colour = "blue")
)

p1 = pw.load_ggplot(p1, figsize=(2,3))
p1.savefig()

## Residual plot

In [ ]:
(
    ggplot(model_values, aes(x = "fitted_values", y = "residuals"))
    + geom_point()
    + geom_smooth(se = False, colour = "red")
)

## Location-Scale plot

In [ ]:
(
    ggplot(model_values, aes(x = "fitted_values", y = "std_resid"))
    + geom_point()
    + geom_smooth(se = False, colour = "red")
)

## Cook's distance

In [ ]:
(
    ggplot(model_values, aes(x = "obs", y = "cooks_d"))
    + geom_point()
    + geom_segment(aes(xend = "obs", yend = 0), colour = "blue")
    + geom_hline(aes(yintercept = 0))
    + geom_hline(aes(yintercept = 4/n_obs), colour = "blue", linetype = "dashed")
)